In [1]:
library(tidyverse)
bugbank_data_dir = "/well/bag/clme1992/bugbank_data/"
ukb_data_dir = "/well/bag/wilson/ukb/"
gwas_wrkdir = "/well/bag/clme1992/saige_pipe_test"


-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.3     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.4     v tibble    3.2.1
v lubridate 1.9.3     v tidyr     1.3.0
v purrr     1.0.2     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::filter() masks stats::filter()
x dplyr::lag()    masks stats::lag()
i Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# At each step of QC, track the change in the infection cases

resolve.symlink <- function(x) {
    y <- Sys.readlink(x)
    if (any(is.na(y))) {
        stop("Could not resolve symlink ", x)
    }
    ifelse(y == "", x, file.path(dirname(x), y))
}

# configuration file
source("~/.saige_pipe.config")

lg <- list()
lg$bd_RDdata_file <- paste0(config$ukb.derived.dir, "/ukb41482.ukb41376.fields.RData")
lg$bd_not_lost2followup_file <- paste0(config$ukb.derived.dir, "/ukb41482.English-not-lost-to-followup-8-April-2020.txt")
lg$bed_sample_qc_file <- paste0(config$ukbdir, "/v2/qc/ukb_sqc_v2.txt")
lg$withdrawn_eid_file <- paste0(config$bbdatadir, "/w53100_2023-04-25.csv")
# Pre-computed eids for the bed-format genotypes
lg$bed_eid_file <- paste0(config$ukb.derived.dir, "/analysis.bed.eids.txt")
# Individuals with first degree relatives
lg$remrels_file <- paste0(config$ukb.derived.dir, "/ukb41482.English-remove-first-degree-relatives.eids.txt")
# panUKB ancestral files
lg$pan_ukb_file <- paste0(config$panukb.dir, "/Files for retman/all_pops_non_eur_pruned_within_pop_pc_covs.tsv")
lg$pan_ukb_bridge_file <- paste0(config$panukb.dir, "/ukb53100bridge31063.txt")

### load input files ###
system.time(load(lg$bd_RDdata_file))
all_eids <- bd[, "f.eid"]
bd_not_lost2followup <- scan(lg$bd_not_lost2followup_file, what = "logical") == "TRUE"
withdrawn_eid <- scan(lg$withdrawn_eid_file)
# Sample QC
bed_sample_qc <- read.csv(lg$bed_sample_qc_file, sep = " ")
# The corresponding eids
bed_eid <- scan(lg$bed_eid_file)
# Convert to bd_eid order
sample_qc <- bed_sample_qc[match(all_eids, bed_eid), ]
# Close (first degree) relatives
remrels <- scan(lg$remrels_file)
# load and match the panukb data
panukb <- read.csv(lg$pan_ukb_file, sep = "\t")[, c("s", "pop")]
panukb_bridge <- read.csv(lg$pan_ukb_bridge_file, sep = " ", header = F)
bridge_matched <- panukb_bridge[match(panukb$s, panukb_bridge[, 2]), ]
panukb$eid <- bridge_matched[, 1]
panukb_matched <- panukb[match(all_eids, panukb$eid), ]

### assign assessment centre data ###
f.assesscentre <- "f.54.0.0"
assess_centre_England <- c(
    11012, 11021, 11011, 11008, 11024, 11020, 11018, 11010, 11016, 
    11001, 11017, 11009, 11013, 11002, 11007, 11014, 10003, 11006, 
    11025, 11026, 11027, 11028 
)
f_assesscentre <- "f.54.0.0"

   user  system elapsed 
 18.088   2.091  20.239 

In [3]:
# get summary for SGSS
sgss_file = paste0(bugbank_data_dir, "ukb_sgss_extract_refined.csv")
sgss = read.csv(sgss_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

# load HES
start_time = Sys.time()
hes_file = paste0(ukb_data_dir, "hes/hesin_diag.latest.txt.gz")
hes = read.table(hes_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)
end_time = Sys.time()
print(paste0("Time taken to load hes: ", end_time - start_time))

[1] "Time taken to load hes: 22.527595281601"


In [46]:
filter_lst <- list()
filter <- rep(T, nrow(bd))
filter_lst[[1]] <- filter
names(filter_lst)[1] <- "all"

filter <- filter & bd_not_lost2followup
filter <- filter & !(all_eids %in% withdrawn_eid)
filter_lst[[2]] <- filter
names(filter_lst)[2] <- "not lost to followup"

filter <- filter & sample_qc$putative.sex.chromosome.aneuploidy == 0
filter_lst[[3]] <- filter
names(filter_lst)[3] <- "no aneuploidy in sex chromosome"

filter <- filter & sample_qc$Submitted.Gender == sample_qc$Inferred.Gender
filter_lst[[4]] <- filter
names(filter_lst)[4] <- "reported sex matches genetic sex"

filter <- filter & sample_qc$het.missing.outliers == 0
filter_lst[[5]] <- filter
names(filter_lst)[5] <- "no het missing outliers"

filter <- filter & sample_qc$excluded.from.kinship.inference == 0
filter_lst[[6]] <- filter
names(filter_lst)[6] <- "not excluded from kinship inference"

filter <- filter & sample_qc$excess.relatives == 0 & is.na(match(all_eids, remrels))
filter_lst[[7]] <- filter
names(filter_lst)[7] <- "no excess nor 1st degree relatives"

filter <- filter & sample_qc$in.Phasing.Input.chr1_22 == 1 & 
    sample_qc$in.Phasing.Input.chrX == 1 &
    sample_qc$in.Phasing.Input.chrXY == 1
filter_lst[[8]] <- filter
names(filter_lst)[8] <- "in phasing input"

filter <- filter & all_eids %in% panukb_matched$eid[which(panukb_matched$pop == "EUR")]
filter_lst[[9]] <- filter
names(filter_lst)[9] <- "in panukb EUR"

filter <- filter & bd[, f_assesscentre] %in% assess_centre_England
filter_lst[[10]] <- filter
names(filter_lst)[10] <- "in assessment centre England"



In [47]:
sgss_filter <- sgss
hes_filter <- hes_infect
ukb_filter <- list()
for (i in 1:length(filter_lst)) {
    filter <- filter_lst[[i]]
    eids_filtered <- unique(all_eids[filter])
    sgss_filter <- sgss_filter[sgss_filter$UKB_EID %in% eids_filtered, ]
    hes_filter <- hes_filter[hes_filter$eid %in% eids_filtered, ]
    sgss_cnt <- length(unique(sgss_filter$UKB_EID))
    hes_cnt <- length(unique(hes_filter$eid))
    ukb_filter[[i]] <- c(length(eids_filtered), sgss_cnt, hes_cnt)
}
ukb_filter <- do.call(rbind, ukb_filter)
rownames(ukb_filter) <- names(filter_lst)

ukb_filter

all,502505,114736,69900
not lost to followup,426755,108226,56142
no aneuploidy in sex chromosome,414121,104782,54100
reported sex matches genetic sex,413957,104733,54067
no het missing outliers,413123,104500,53948
not excluded from kinship inference,413115,104498,53948
no excess nor 1st degree relatives,390900,98586,50918
in phasing input,390739,98537,50888
in panukb EUR,339953,87294,43750
in assessment centre England,339953,87294,43750


### Pathogen filtering - SGSS

In [7]:
# for SGSS
sgss_tax_file <- paste0(bugbank_data_dir, "bb_pathogen_taxonomy_13032023.tsv")
sgss_tax <- read.table(sgss_tax_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

# ------------------------------------------------------------
# create a dictionary from origin_name to most specific taxa
# ------------------------------------------------------------

# sanity check: no NA nor duplicated origin_name
sum(is.na(sgss_tax$origin_name))
sum(duplicated(sgss_tax$origin_name))
origin_name_to_tax <- list()

# for each origin_name, find the most specific tax

# get the tax levels
tax_levs <- colnames(sgss_tax)[-ncol(sgss_tax)]
# remove the last column (which is the origin_name) for later use
sgss_tax_only <- sgss_tax[, -ncol(sgss_tax)]

for (i in 1:nrow(sgss_tax)) {
    origin_name <- sgss_tax$origin_name[i]
    # remove any NA and the last column (which is the origin_name)
    tax <- sgss_tax_only[i, ]
    tax_lev <- tax_levs[!is.na(tax)]
    tax <- tax[!is.na(tax)]
    specific_tax <- tail(tax, 1)
    specific_tax_lev <- tail(tax_lev, 1)
    origin_name_to_tax[[origin_name]] <- c(specific_tax, specific_tax_lev)
}



[1] 0

[1] 0

In [114]:
# filter1: starting from all
filter_lst <- list()
filter <- rep(T, nrow(sgss))

filter_lst[[1]] <- filter
names(filter_lst)[1] <- "all"

# filter2: only include individuals that passed the QC
filter_lst[[2]] <- filter & sgss$UKB_EID %in% eids_filtered
names(filter_lst)[2] <- "individual that passed QC"

# filter3: only include infection that have a pathogen label in species level
sgss$tax_lev <- map_chr(sgss$ORGANISM_SPECIES_NAME, function(origin_name) {
    tax_lev <- origin_name_to_tax[[origin_name]][2]
})
filter_lst[[3]] <- filter_lst[[2]] & sgss$tax_lev == "species"
names(filter_lst)[3] <- "species level pathogen"

# filter4: only include pathogen that have infection cases > 100
sgss$tax <- map_chr(sgss$ORGANISM_SPECIES_NAME, function(origin_name) {
    species_name <- origin_name_to_tax[[origin_name]][1]
})
species_name_qc <- unique(sgss$tax[filter_lst[[3]]])
# get the frequency of each species by the number of unique individuals
species_name_freq <- map_int(species_name_qc, function(species_name) {
    cur_filter <- (sgss$tax == species_name) & filter_lst[[3]]
    cur_eids <- sgss$UKB_EID[cur_filter]
    return(length(unique(cur_eids)))
})
names(species_name_freq) <- species_name_qc
species_name_gwas <- names(species_name_freq)[which(species_name_freq > 100)]
filter_lst[[4]] <- filter_lst[[3]] & (sgss$tax %in% species_name_gwas)
names(filter_lst)[4] <- "pathogen with > 100 cases"

sgss_filter_table <- list()
for(i in 1:length(filter_lst)) {
    filter <- filter_lst[[i]]
    record_cnt <- sum(filter, na.rm = TRUE)
    ind_cnt <- length(unique(sgss$UKB_EID[filter]))
    tax_cnt <- length(unique(sgss$tax[filter]))
    sgss_filter_table[[i]] <- c(record_cnt, ind_cnt, tax_cnt)
}
sgss_filter_table <- do.call(rbind, sgss_filter_table)
rownames(sgss_filter_table) <- names(filter_lst)
colnames(sgss_filter_table) <- c("record_cnt", "ind_cnt", "tax_cnt")
sgss_filter_table


,record_cnt,ind_cnt,tax_cnt
all,350699,114737,526
individual that passed QC,265114,87294,479
species level pathogen,204019,73756,382
pathogen with > 100 cases,200529,72893,31


In [97]:
# sanity check 1
# see if the resulting number of tax is the same as the number of GWAS
gwas_wrkdir <- "/well/bag/clme1992/saige_pipe_test"
gwas_files <- list.files(gwas_wrkdir, pattern = "summary.05062023_sgss_species.*all.txt.gz", full.names = TRUE)
gwas_files_species <- gsub(".*summary.05062023_sgss_species.", "", gwas_files)
gwas_files_species <- gsub(".species.all.txt.gz", "", gwas_files_species)
gwas_files_species <- gsub("_", " ", gwas_files_species)
print(length(gwas_files_species))
print(setdiff(gwas_files_species, species_name_gwas))
print(setdiff(species_name_gwas, gwas_files_species))

[1] 36
[1] "Candida parapsilosis" "Micrococcus luteus"   "Proteus vulgaris"    
[4] "Pseudomonas putida"   "Salmonella enterica" 
character(0)


In [107]:
# sanity check 2
# see if number of infection is the same for E.coli from the phenotype file
ecoli_phe_file <- paste0(gwas_wrkdir, "/ukb41482.bd.gwas-pheno.05062023_sgss_species.Escherichia_coli.species.all.txt.gz")
ecoli_phe <- read.table(ecoli_phe_file, sep = "\t", header = T, stringsAsFactors = F)

In [112]:
table(ecoli_phe$pheno)
species_name_freq[which(species_name_freq > 100)]


     0      1 
267190  51563 

named integer(0)

### Pathogen filtering - HES

In [29]:
# For HES

# load hes_diag
start_time = Sys.time()
hes_file = paste0(ukb_data_dir, "hes/hesin_diag.latest.txt.gz")
hes = read.table(hes_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)
end_time = Sys.time()
print(paste0("Time taken to load hes: ", end_time - start_time))

[1] "Time taken to load hes: 18.9089567661285"


In [32]:
# load icd10 data 
icd10_desc_file <- paste0(bugbank_data_dir, "pathogen_to_unique_icd10.tsv")
icd10_desc <- read.table(icd10_desc_file, sep = "\t", header = TRUE, stringsAsFactors = FALSE)

# subset hes to only infection related
infect_icd10_codes <- unique(unlist(strsplit(icd10_desc$icd10, split = ",")))
hes_infect <- hes[hes$diag_icd10 %in% infect_icd10_codes, ]

# create a dictionary of ICD-10 to pathogen mapping
icd_to_pathogen = list()
for (i in 1:nrow(icd10_desc)) {
  cur_icd10s = unlist(strsplit(icd10_desc$icd10[i], ","))
  for (icd10 in cur_icd10s) {
    # check if the icd10 is already in the dictionary
    # if so, report an error
    if (icd10 %in% names(icd_to_pathogen)) {
      print(paste0("Error: ", icd10, " already in the dictionary"))
    }
    icd_to_pathogen[[icd10]] = c(icd10_desc$org_name[i], icd10_desc$tax_lev[i])
  }
}

In [82]:
# filter1: starting from all
filter_lst <- list()
filter <- rep(T, nrow(hes_infect))
filter_lst[[1]] <- filter
names(filter_lst)[1] <- "all"

# filter2: only include individuals that passed the QC
filter_lst[[2]] <- filter & hes_infect$eid %in% eids_filtered
names(filter_lst)[2] <- "individual that passed QC"

#filter3: only include infection that have a pathogen label in species level
hes_infect$tax_lev <- map_lgl(hes_infect$diag_icd10, function(icd10) {
  tax_lev <- icd_to_pathogen[[icd10]][2]
  return(tax_lev == "species")
})
filter_lst[[3]] <- filter_lst[[2]] & hes_infect$tax_lev
names(filter_lst)[3] <- "species level pathogen"

# filter4: only include pathogen that have infection cases > 100
hes_infect$tax <- map_chr(hes_infect$diag_icd10, function(icd10) {
    species_name <- icd_to_pathogen[[icd10]][1]
    return(species_name)  
})
species_name_qc <- unique(hes_infect$tax[filter_lst[[3]]])
# get the frequency of each species by the number of unique individuals
species_name_freq <- map_int(species_name_qc, function(species_name) {
    cur_filter <- (hes_infect$tax == species_name) & filter_lst[[3]]
    cur_eids <- hes_infect$eid[cur_filter]
    return(length(unique(cur_eids)))
})
names(species_name_freq) <- species_name_qc
species_name_gwas <- names(species_name_freq)[which(species_name_freq > 100)]
filter_lst[[4]] <- filter_lst[[3]] & (hes_infect$tax %in% species_name_gwas)
names(filter_lst)[4] <- "pathogen with > 100 cases"

# gather the filter into a table
hes_filter_table <- list()
for(i in 1:length(filter_lst)) {
    filter <- filter_lst[[i]]
    record_cnt <- sum(filter, na.rm = TRUE)
    ind_cnt <- length(unique(hes_infect$eid[filter]))
    tax_cnt <- length(unique(hes_infect$tax[filter]))
    hes_filter_table[[i]] <- c(record_cnt, ind_cnt, tax_cnt)
}
hes_filter_table <- do.call(rbind, hes_filter_table)
rownames(hes_filter_table) <- names(filter_lst)
colnames(hes_filter_table) <- c("record_cnt", "ind_cnt", "tax_cnt")
hes_filter_table

,record_cnt,ind_cnt,tax_cnt
all,188788,69900,155
individual that passed QC,112022,43750,140
species level pathogen,30685,15141,78
pathogen with > 100 cases,29450,14540,17


In [83]:
# sanity check, see if the resulting number of tax is the same as the number of GWAS
gwas_wrkdir <- "/well/bag/clme1992/saige_pipe_test"
gwas_files <- list.files(gwas_wrkdir, pattern = "summary.05062023_hes_species.*.txt.gz", full.names = TRUE)
gwas_files_species <- gsub(".*summary.05062023_hes_species.regenie.", "", gwas_files)
gwas_files_species <- gsub(".species.txt.gz", "", gwas_files_species)
gwas_files_species <- gsub("_", " ", gwas_files_species)
print(length(gwas_files_species))
print(setdiff(gwas_files_species, species_name_gwas))
print(setdiff(species_name_gwas, gwas_files_species))


[1] 20
[1] "Hepatovirus A"          "Plasmodium falciparum"  "Pneumocystis jirovecii"
character(0)


In [75]:
pheno_rds_file <- paste0(gwas_wrkdir, "/log.ukb41482.gwas-stratify.05062023_hes_species_regenie.rds")
pheno_rds <- readRDS(pheno_rds_file)


In [84]:
species_name_freq[which(species_name_freq > 100)]

Klebsiella pneumoniae             Escherichia coli 
                         713                         4456 
  Mycobacterium tuberculosis        Staphylococcus aureus 
                         147                         2839 
               Enterovirus C         Human papillomavirus 
                         149                         1415 
      Human orthopneumovirus     Human alphaherpesvirus 3 
                         105                          842 
unidentified influenza virus       Haemophilus influenzae 
                        1179                          431 
         Helicobacter pylori     Streptococcus pneumoniae 
                        1961                          547 
    Clostridioides difficile            Hepatitis B virus 
                         770                          172 
               Hepacivirus C Human immunodeficiency virus 
                         255                          115 
     Enterobius vermicularis 
                         130

In [79]:
names(pheno_rds)
pheno_rds$n.cases

[1] "args"                   "stem"                   "username"              
 [4] "srcdir"                 "joint_phenotypes_file"  "log_rds_outfile"       
 [7] "data_outfileprefix"     "data_outfiles"          "pheno_colnames"        
[10] "table_filtered_pheno"   "na_counts_columns_data" "n.cases"

$Escherichia_coli.species
[1] 5144

$Klebsiella_pneumoniae.species
[1] 844

$Haemophilus_influenzae.species
[1] 489

$Helicobacter_pylori.species
[1] 2513

$Mycobacterium_tuberculosis.species
[1] 288

$Staphylococcus_aureus.species
[1] 3238

$Streptococcus_pneumoniae.species
[1] 631

$Clostridioides_difficile.species
[1] 869

$Human_orthopneumovirus.species
[1] 119

$unidentified_influenza_virus.species
[1] 1370

$Enterovirus_C.species
[1] 174

$Echovirus.species
[1] 48

$Hepatovirus_A.species
[1] 98

$Hepacivirus_C.species
[1] 377

$Hepatitis_B_virus.species
[1] 299

$Human_immunodeficiency_virus.species
[1] 176

$Human_alphaherpesvirus_3.species
[1] 967

$Human_papillomavirus.species
[1] 1594

$Pneumocystis_jirovecii.species
[1] 83

$Plasmodium_falciparum.species
[1] 149

$Enterobius_vermicularis.species
[1] 140

$ancestry_panukb
[1] 0